In [ ]:
# Work flow:
# Open video file
# Extract frame from video (Extract x number of frames from video?)
# Hash frame
# Compare hash value with other hash values
# Cluster videos based on similarity of hash values

In [1]:
from adjusted_rand_index import rand_index
from sklearn.cluster import KMeans
import os
import io
import math
import PIL
import numpy
from PIL import Image
from threading import Thread
import numpy as np
from datasketch import MinHash
from subprocess import call

DATASET_FOLDER = 'videos/'
IMAGE_FOLDER = "images/"
NUM_OF_TEST_VIDEOS = 50
NUM_OF_CLUSTERS = 3

# Empty dictionary to store video name and hash values
hash_values = {}

In [2]:
def hash_all_videos():
    for filename in sorted(os.listdir(DATASET_FOLDER))[:NUM_OF_TEST_VIDEOS]:
        call("""ffmpeg -i """ + DATASET_FOLDER + filename + """ -vf "select='eq(pict_type,PICT_TYPE_I)'" -vsync vfr images/image_%d.png""", shell=True)
        hash_values[filename] =  combined_hash_value(IMAGE_FOLDER)
        delete_images()

In [3]:
def delete_images():
    call("rm images/*.png", shell=True)

In [4]:
def convert_image_to_hash(img_path):
    # Converte img to grayscale & resize img
    img = Image.open(img_path).convert('LA').resize((10,10), PIL.Image.ANTIALIAS)
    numpy_array = numpy.array(img)
    min_hash = MinHash(num_perm=128)
    # Hash img
    for val in numpy_array:
        min_hash.update(val)
        
    return min_hash

In [5]:
def merge_hashed_values(frames):
    first_frame = frames.pop(0)
    
    for frame in frames:
        first_frame.merge(frame)
    
    return first_frame

In [9]:
def combined_hash_value(imagefolder):
    hashed_images = []

    for filename in sorted(os.listdir(imagefolder)):
        hashed_images.append(convert_image_to_hash(imagefolder+filename))

    return merge_hashed_values(hashed_images)

In [10]:
def prepare_x_values(hashed_values):
    value_list = []

    for key in hash_values:
        value_list.append(hashed_values[key].hashvalues)
        
    return value_list

In [ ]:
value_list = []

for key in hash_values:
    value_list.append(hash_values[key].hashvalues)
    
X = np.array(value_list)
kmeans = KMeans(n_clusters=NUM_OF_CLUSTERS, random_state=0).fit(X)

In [ ]:
for key in hash_values:    
    if (hash_values[key].hashvalues == list(value_list[0])).all():
        print(key)

In [ ]:
for index in range(0,NUM_OF_CLUSTERS):
    cluster = []
    cluster_index = np.where(kmeans.labels_==index)
    for i in cluster_index:
        for key in hash_values:
            if hash_values[key].hashvalues == value_list[i]:
                cluster.append
    
    

In [7]:
hash_all_videos()

In [8]:
len(hash_values)

50